## Policing in Schools
A look at whether school policing can predict graduation rates.

By Onel Abreu, Alexander Roche, Sabrina Sedovic

In [1]:
import pandas as pd
import pipeline as pipeline

school_data = pd.read_csv("school_data.csv")
school_data.head(10)

,Unnamed: 0,Unnamed: 0.1,Year,Dept/Unit Number,Bilingual_N,Bilingual_P,Free_Lunch_N,Free_Lunch_P,IEP_N,IEP_P,...,# of Unique Students Receiving OSS,% of Unique Students Receiving OSS,Average Length of OSS,# of Police Notifications,% of Misconducts Resulting in a Police Notification,Police Notifications per 100 Students,# of Unique Students Receiving Police Notification,% of Unique Students Receiving Police Notification,# of Students Expelled,Expulsions per 100 Students
0,64,64,2011,53011,2.0,0.001391,1393.0,0.968707,302.0,0.210014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,65,65,2011,53021,7.0,0.004342,1501.0,0.931141,156.0,0.096774,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,66,66,2011,45231,1.0,0.007937,110.0,0.873016,7.0,0.055556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,67,67,2011,47021,17.0,0.020556,454.0,0.548972,91.0,0.110036,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,68,68,2011,49141,NaN,NaN,304.0,0.921212,61.0,0.184848,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,69,69,2011,49141,NaN,NaN,304.0,0.921212,61.0,0.184848,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,70,70,2011,53041,55.0,0.038814,1276.0,0.900494,169.0,0.119266,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,71,71,2011,49151,7.0,0.021807,279.0,0.869159,19.0,0.059190,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,72,72,2011,70020,3.0,0.003304,303.0,0.333700,58.0,0.063877,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,73,73,2011,53051,59.0,0.098333,587.0,0.978333,88.0,0.146667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
school_data = school_data.drop(columns = ["Unnamed: 0", "Unnamed: 0.1", "Dept/Unit Number",
                                          "Job Title_Career Counseling Manager",
                                          "Job Title_Chief Safety & Security OFF",
                                          "Job Title_Director of Counseling",
                                          "Job Title_Director, School Counseling & Post Secondary Advising",
                                          "Job Title_Flex Team Security Officer",
                                          "Job Title_HS Counseling Specialist", 
                                          "Job Title_K‐8 Counseling Specialist",
                                          "Job Title_Manager-Secdry Schl Counselng", "Notes",
                                          "School_x", "School ID", "pos_name_0", "School Name_x", "Unit_2",
                                          "Unit_Number", "School_y", "School Name_y", "School Year"])

### Data Cleaning

 ### Initial Data Analysis

### Pipeline

### Evaluation